In [12]:
import pandas as pd
import faiss

import tensorflow_hub as hub

from pprint import pprint

In [2]:
# IMPORTANT - used to find doc index
tokens_chunk_size = 512

In [15]:
legal_docs = pd.read_csv("../data/legal_text_classification.csv")
index = faiss.read_index("non_chunked_court_text.index")

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [23]:
query1 = "claims related to shoe companies"
#query1 = "claims involving large sums of money"

# Encode the query into an embedding
query_embedding = embed([query1]).numpy()

# Search for similar embeddings in the index
D, I = index.search(query_embedding, k=5)  # Retrieve the top 5 similar documents

# Print the indices and distances of the similar documents
print("Top 5 similar documents:")
indices = []
for i, d in zip(I[0], D[0]):
    print(f"Index: {i}, Similarity Score: {1 - d}")
    indices.append(i)

Top 5 similar documents:
Index: 5748, Similarity Score: -0.34309840202331543
Index: 22063, Similarity Score: -0.3481215238571167
Index: 6910, Similarity Score: -0.3489185571670532
Index: 10535, Similarity Score: -0.3654189109802246
Index: 22065, Similarity Score: -0.37540555000305176


In [25]:
idx = 6910

pprint (legal_docs.iloc[idx]["case_title"])
pprint (legal_docs.iloc[idx]["case_text"])

'World Brands Management Pty Ltd v Cube Footwear Pty Ltd [2004] FCA 769'
('The above findings are based not only on the evidence in the proceedings, '
 'but on commonsense and judicial experience of the process people typically '
 'engage in when purchasing footwear. In World Brands Management Pty Ltd v '
 'Cube Footwear Pty Ltd [2004] FCA 769 at [7] (albeit in a passing off '
 'context) Heerey J said: Turning to the TPA and passing-off claims, I do not '
 'agree that the decision of Goldberg J in Dr Martens Australia Pty Ltd v '
 'Figgins Holdings Pty Ltd (1999) 44 IPR 281 establishes some rule of law as '
 'to the circumstances which pertain in the retail footwear trade. '
 "Nevertheless, his Honour's observations are valuable in another way. They "
 "light up and provoke one's memories from personal experience as to the way "
 'in which shoes are bought by retail consumers. As counsel for the '
 'respondents pointed out, shoes are usually not a quick impulsive purchase of '
 'someth